In [2]:
# Pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.api import VAR
import statsmodels.api as sm
import yfinance as yf

In [3]:
### External Data Sources (optional) ###
import ipeadatapy as ipea  # For accessing IPEA data (if needed)

In [4]:
ipea.metadata()

,CODE,NAME,COMMENT,LAST UPDATE,BIG THEME,SOURCE ACRONYM,SOURCE,SOURCE URL,FREQUENCY,MEASURE,UNIT,SERIES STATUS,THEME CODE,COUNTRY,NUMERICA
0,ABATE_ABPEAV,Abate - aves - peso das carcaças,O abate de animais é mensurado por sua quantid...,2025-03-18T13:00:00.673-03:00,Macroeconômico,IBGE/Coagro,Instituto Brasileiro de Geografia e Estatístic...,www.ibge.gov.br,Anual,Tonelada,mil,A,1,BRA,True
1,ABATE_ABQUBV,Abate - bovinos - quantidade,O abate de animais é mensurado por sua quantid...,2025-03-18T13:00:00.673-03:00,Macroeconômico,IBGE/Coagro,Instituto Brasileiro de Geografia e Estatístic...,www.ibge.gov.br,Anual,Cabeça,mil,A,1,BRA,True
2,ABATE12_ABPEVA12,Abate - vacas - peso das carcaças,O abate de animais é mensurado por sua quantid...,2025-03-18T12:26:00.49-03:00,Macroeconômico,IBGE/Coagro,Instituto Brasileiro de Geografia e Estatístic...,www.ibge.gov.br,Mensal,Tonelada,mil,I,1,BRA,True
3,ABATE12_ABQUBO12,Abate - bois - quantidade,O abate de animais é mensurado por sua quantid...,2025-03-18T12:26:00.49-03:00,Macroeconômico,IBGE/Coagro,Instituto Brasileiro de Geografia e Estatístic...,www.ibge.gov.br,Mensal,Cabeça,mil,A,1,BRA,True
4,ABATE12_ABQUBV12,Abate - bovinos - quantidade,O abate de animais é mensurado por sua quantid...,2025-03-18T12:26:00.49-03:00,Macroeconômico,IBGE/Coagro,Instituto Brasileiro de Geografia e Estatístic...,www.ibge.gov.br,Mensal,Cabeça,mil,A,1,BRA,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2821,PNADCT_TXPARTCUF_SI,Taxa de participação - sem instrução ou equiva...,"Taxa de participação na força de trabalho, na ...",2025-02-18T12:24:51.957-03:00,Social,IBGE/PNAD Contínua,Instituto Brasileiro de Geografia e Estatístic...,http://www.ibge.gov.br/home/estatistica/indica...,Trimestral,(%),None,None,110,None,True
2822,PNAD_IAGRV,Domicílios com insegurança alimentar grave,Distribuição percentual dos domicílios de acor...,2024-06-20T10:44:01.677-03:00,Social,IBGE/PNAD Contínua,Instituto Brasileiro de Geografia e Estatístic...,http://www.ibge.gov.br/home/estatistica/indica...,Decenal,(%),None,None,111,None,True
2823,PNAD_IALEV,Domicílios com insegurança alimentar leve,Distribuição percentual dos domicílios de acor...,2024-06-20T10:44:01.68-03:00,Social,IBGE/PNAD Contínua,Instituto Brasileiro de Geografia e Estatístic...,http://www.ibge.gov.br/home/estatistica/indica...,Decenal,(%),None,None,111,None,True
2824,PNAD_IAMOD,Domicílios com insegurança alimentar moderada,Distribuição percentual dos domicílios de acor...,2024-06-20T10:44:01.683-03:00,Social,IBGE/PNAD Contínua,Instituto Brasileiro de Geografia e Estatístic...,http://www.ibge.gov.br/home/estatistica/indica...,Decenal,(%),None,None,111,None,True


In [5]:
ipca = ipea.timeseries('PRECOS12_IPCA12')

ipca = ipca.rename(columns={
    'RAW DATE': 'date',
    'VALUE (-)': 'IPCA',
    'CODE': 'code',
    'DAY': 'day',
    'MONTH': 'month',
    'YEAR': 'year'
})

In [6]:
ipca

,code,date,day,month,year,IPCA
DATE,,,,,,
1979-12-01,PRECOS12_IPCA12,1979-12-01T00:00:00-02:00,1,12,1979,7.618300e-09
1980-01-01,PRECOS12_IPCA12,1980-01-01T00:00:00-02:00,1,1,1980,8.122300e-09
1980-02-01,PRECOS12_IPCA12,1980-02-01T00:00:00-02:00,1,2,1980,8.497300e-09
1980-03-01,PRECOS12_IPCA12,1980-03-01T00:00:00-03:00,1,3,1980,9.010400e-09
1980-04-01,PRECOS12_IPCA12,1980-04-01T00:00:00-03:00,1,4,1980,9.486700e-09
...,...,...,...,...,...,...
2024-11-01,PRECOS12_IPCA12,2024-11-01T00:00:00-03:00,1,11,2024,7.063770e+03
2024-12-01,PRECOS12_IPCA12,2024-12-01T00:00:00-03:00,1,12,2024,7.100500e+03
2025-01-01,PRECOS12_IPCA12,2025-01-01T00:00:00-03:00,1,1,2025,7.111860e+03


In [9]:
import requests

url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.24363/dados"
params = {
    "formato": "json",
    "dataInicial": "01/01/2000"
}

response = requests.get(url, params=params)
data = response.json()

In [ ]:
data

In [ ]:


# Importante: você usou `ipeadatar` e `GetBCBData` no R
# Em Python, não temos pacotes oficiais equivalentes
# Então para IBC-Br (ID 24363) e Commodity Price Index (ID 27574), vamos simular ou usar fontes como BCB API ou outras bases.

# Aqui vou assumir que você já tem os dados. Se não, depois te ajudo a puxar!

# ------------------------
# Simulando o carregamento de séries
# ------------------------

# Para exemplo:
# Você pode usar APIs como SGS ou planilhas baixadas.

# Digamos que você já carregou:
# ibc = pd.read_csv('ibc.csv')
# comm = pd.read_csv('comm.csv')
# ipca = pd.read_csv('ipca.csv')
# selic = pd.read_csv('selic.csv')
# cambio = pd.read_csv('cambio.csv')

# Aqui é a estrutura correta para continuar:

# Suponha que as séries estejam como DataFrames com colunas ['date', 'value']

# Transformações

# 1. Log dos índices
ibc['log_value'] = np.log(ibc['value'])
comm['log_value'] = np.log(comm['value'])
cambio['log_value'] = np.log(cambio['value'])

# 2. IPCA: Transformar para variação % a/a
ipca['ipca_12m'] = (ipca['value'] / ipca['value'].shift(12) - 1) * 100

# 3. Janela 2003-01 até 2025-02
start_date = '2003-01-01'
end_date = '2025-02-01'

# Filtrar
ibc = ibc[(ibc['date'] >= start_date) & (ibc['date'] <= end_date)]
comm = comm[(comm['date'] >= start_date) & (comm['date'] <= end_date)]
ipca = ipca[(ipca['date'] >= start_date) & (ipca['date'] <= end_date)]
selic = selic[(selic['date'] >= start_date) & (selic['date'] <= end_date)]
cambio = cambio[(cambio['date'] >= start_date) & (cambio['date'] <= end_date)]

# 4. Real interest rate
selic['selic_real'] = (1 + selic['value']/100) / (1 + ipca['ipca_12m']/100) - 1

# 5. Construir base unificada
base = pd.DataFrame({
    'tscomm': comm['log_value'].values,
    'tsreal': selic['selic_real'].values,
    'tscambio': cambio['log_value'].values,
    'tsibc': ibc['log_value'].values,
    'tsipca': ipca['ipca_12m'].values
}, index=pd.to_datetime(ibc['date']))

# ------------------------
# VAR
# ------------------------

# Estimando o VAR com p=15
model = VAR(base)
results = model.fit(15, trend='ct')  # tipo 'both' = constante + tendência

# ------------------------
# SVAR com restrições de sinal
# ------------------------

# No Python não existe um pacote igual ao `bsvarSIGNs`
# Mas podemos fazer uma identificação estrutural manual
# Vou usar SVAR com `A` restrictions do `statsmodels`:

from statsmodels.tsa.vector_ar.svar_model import SVAR

# Matriz de sinais (equivalente ao que você criou no mat_sig)
# Atenção: aqui normalmente você define restrições de short-run ou contemporâneas

# Exemplo:
# Matriz A para short-run restrictions
A = np.array([
    [np.nan, 0,    np.nan, 0,    0],
    [np.nan, np.nan, np.nan, np.nan, np.nan],
    [0,    np.nan, np.nan, 0,    0],
    [0,    np.nan, np.nan, np.nan, 0],
    [0,    0,    np.nan, 0, np.nan]
])

svar = SVAR(results, svar_type='A', A=A)

svar_results = svar.fit()

# ------------------------
# Impulse Response Functions (IRF)
# ------------------------

irf = svar_results.irf(34)

# Plot
fig = irf.plot(orth=False, signif=0.68)
plt.tight_layout()
plt.show()
